In [1]:
import pandas as pd
import os 
import numpy as np
import utm

#Files
os.chdir('/home/odd1/Desktop/ecology_project/jsdm1.0/EcologyDataScienceProject/dataPreprocessing/')
path = "/home/odd1/Desktop/ecology_project/jsdm1.0/EcologyDataScienceProject/dataPreprocessing/"
daymetFile = "climate_daymet.csv"
fiaFile = "climate_fia.csv"
task3TrainingFile = "training.csv"
task3TestingFile = "testing.csv"
task2Itc = "itc_attributes_train.csv"
crownCoords = "crown_coords.csv"
latBand = 17
lonBand = 'R'

def utmToLatLon(x):
   return utm.to_latlon(x[0], x[1], latBand, lonBand)

def utmFromLatLon(x):
   temp = utm.from_latlon(x[0], x[1])
   temp = (temp[0], temp[1])
   return temp

def l2(pos1, pos2):
   return np.sqrt(((pos1[0] - pos2[0])**2) + ((pos1[1] - pos2[1])**2))

def nearestFIAplot(idx):
   closestFiaPlotIdx = -1
   closestDist = float('inf')

   itcEN = (itcDf.iloc[idx, 4], itcDf.iloc[idx, 5])

   for fiaIdx, row in fiaDf.iterrows():
      fiaEN = utmFromLatLon((row['lat'], row['lon'])) #utmFromLatLon((fiaDf.iloc[numRow, 4], fiaDf.iloc[numRow, 5]))
      dist = l2(fiaEN, itcEN)
      
      if (dist < closestDist):
         closestDist = dist
         closestFiaPlotIdx = fiaIdx
   return fiaDf.loc[closestFiaPlotIdx, 'plot']

def geoToUtm(geoString):
   temp = tuple(map(float, geoString.strip("c()").split(",")))
   return temp
         
fiaDf = pd.read_csv(os.path.join(fiaFile))
trainDf = pd.read_csv(os.path.join(task3TrainingFile))
testDf = pd.read_csv(os.path.join(task3TestingFile))
itcDf = pd.read_csv(os.path.join(crownCoords))
daymetDf = pd.read_csv(os.path.join(daymetFile))

In [2]:
fiaDf = fiaDf[(fiaDf.statecd == 12) & (fiaDf.countycd == 107)]
print(fiaDf.columns)

Index(['statecd', 'unitcd', 'countycd', 'plot', 'lat', 'lon', 'slope',
       'aspect', 'elev', 'carbon_soil_org', 'watercd', 'physclcd', 'invyr'],
      dtype='object')


In [3]:
tempDf = itcDf['geometry'].apply(geoToUtm).apply(pd.Series)
tempDf.columns = ['ITC_E', 'ITC_N']
itcDf = itcDf.join(tempDf)
print(itcDf.columns)

Index(['crown_id', 'confidence', 'Plot_ID', 'geometry', 'ITC_E', 'ITC_N'], dtype='object')


In [4]:
daymetDf = daymetDf[(daymetDf.statecd == 12) & (daymetDf.countycd == 107)]
daymetDf = daymetDf.drop(['countycd', 'unitcd', 'statecd', 'Unnamed: 0'], axis = 1)
print(daymetDf.columns)

Index(['plot', 'daylength_3', 'daylength_4', 'daylength_10', 'prec_6',
       'prec_7', 'rad_8', 'tmax_8', 'tmin_1'],
      dtype='object')


In [5]:
newCol = []
for idx, row in itcDf.iterrows():
   newCol.append(nearestFIAplot(idx))

itcDf['closestfia'] = newCol
print(itcDf.columns)

Index(['crown_id', 'confidence', 'Plot_ID', 'geometry', 'ITC_E', 'ITC_N',
       'closestfia'],
      dtype='object')


In [6]:
tempDf = itcDf.merge(fiaDf, left_on='closestfia', right_on='plot')
tempDf = tempDf.merge(daymetDf, on='plot')
print(tempDf.columns)

Index(['crown_id', 'confidence', 'Plot_ID', 'geometry', 'ITC_E', 'ITC_N',
       'closestfia', 'statecd', 'unitcd', 'countycd', 'plot', 'lat', 'lon',
       'slope', 'aspect', 'elev', 'carbon_soil_org', 'watercd', 'physclcd',
       'invyr', 'daylength_3', 'daylength_4', 'daylength_10', 'prec_6',
       'prec_7', 'rad_8', 'tmax_8', 'tmin_1'],
      dtype='object')


In [7]:
tempTrainDf = trainDf.merge(tempDf, on='crown_id')
tempTestDf = testDf.merge(tempDf, on='crown_id')

In [7]:
tempTrainDf = trainDf.merge(tempDf, on='crown_id')
tempTestDf = testDf.merge(tempDf, on='crown_id')

finalTrainDf = tempTrainDf.drop(['confidence', 'Plot_ID', 'geometry', 'ITC_E', 'ITC_N', 'closestfia', 'statecd', 'unitcd', 'countycd', 'plot', 'invyr'], axis=1)
finalTestDf = tempTestDf.drop(['confidence', 'Plot_ID', 'geometry', 'ITC_E', 'ITC_N', 'closestfia', 'statecd', 'unitcd', 'countycd', 'plot', 'invyr'], axis=1)

print(finalTrainDf.columns)
print(finalTestDf.columns)

Index(['crown_id', 'band_1', 'band_2', 'band_3', 'band_4', 'band_5', 'band_6',
       'band_7', 'band_8', 'band_9',
       ...
       'watercd', 'physclcd', 'daylength_3', 'daylength_4', 'daylength_10',
       'prec_6', 'prec_7', 'rad_8', 'tmax_8', 'tmin_1'],
      dtype='object', length=443)
Index(['crown_id', 'band_1', 'band_2', 'band_3', 'band_4', 'band_5', 'band_6',
       'band_7', 'band_8', 'band_9',
       ...
       'watercd', 'physclcd', 'daylength_3', 'daylength_4', 'daylength_10',
       'prec_6', 'prec_7', 'rad_8', 'tmax_8', 'tmin_1'],
      dtype='object', length=443)


In [8]:
finalTrainDf.to_csv('trainPlus.csv', index=False, header=True)
finalTestDf.to_csv('testPlus.csv', index=False, header=True)

In [9]:
print(finalTrainDf.shape)
print(finalTestDf.shape)

(6831, 443)
(2625, 443)
